In [ ]:
!pip install gradio

In [ ]:
from joblib import dump, load
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from nltk.tokenize import word_tokenize

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if isinstance(X, pd.Series):
            return X.apply(self.preprocess_text)
        else:  # handle other iterables
            return [self.preprocess_text(text) for text in X]
    
    def preprocess_text(self, text):
        tokens = word_tokenize(text)
        tokens = [self.lemmatizer.lemmatize(token.lower()) for token in tokens if token.isalnum()]
        tokens = [token for token in tokens if token not in self.stop_words]
        return " ".join(tokens)

In [ ]:
PATH = r""

loaded_pipeline = load(PATH)

In [ ]:
def senti_alz(input_text):
    prediction = loaded_pipeline.predict([input_text])
    # print(f"The sentiment of the input sentence is: {prediction[0]}")
    confidence_scores = loaded_pipeline.predict_proba([input_text])

    
    return prediction[0],confidence_scores[0]

In [ ]:
import gradio as gr


# Create a Gradio interface with a submit button
iface = gr.Interface(
    fn=senti_alz,
    inputs=gr.Textbox(),  # Gradio Image component takes input as numpy array
    outputs=[gr.Textbox(label="Sentiment"), gr.Textbox(label="Confidence Scores")],
    live=False,
    title="SENTIMENTZZ",
    description="Write the tweet",
    allow_flagging="never",
    theme='Ajaxon6255/Emerald_Isle'

)

# Launch the Gradio interface
iface.launch()